In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math

In [39]:
def haversin(x):
    return math.sin(x)**2

def inverse_haversin(x):
    return math.asin(math.sqrt(x))

def great_circle_dist(a, b, c, d):
    haversin_d_over_R = haversin(b - a) + math.cos(a)*math.cos(b)*haversin(d - c)
    d_over_R = inverse_haversin(haversin_d_over_R)
    d = d_over_R * 6371
    return d

# reference
#https://undergroundmathematics.org/trigonometry-compound-angles/the-great-circle-distance

In [75]:
def mutate_path(path, mutation_rate):
    mutated_path = path.copy()
    for i in range(len(path)):
        rand_num = np.random.uniform(0, 1)
        if rand_num <= mutation_rate:
            idx = int(np.random.choice(len(mutated_path), 1))
            mutated_path[i], mutated_path[idx] = mutated_path[idx], mutated_path[i]
    return mutated_path

In [41]:
def travel_dist(path, adj_matrix):
    total_dist = 0
    for idx in range(len(path) - 1):
        total_dist += adj_matrix[path[idx], path[idx+1]]
    return total_dist

In [42]:
scale_factor = 10**7
def fitness_score(path, adj_matrix):
    return 1/(travel_dist(path, adj_matrix)) * scale_factor

In [43]:
def roulette(sorted_population, adj_matrix):
    total_fs = sum(map(lambda x: fitness_score(x, adj_matrix), sorted_population))
    random_fs = random.uniform(0, total_fs)
    for i in range(len(sorted_population)):
        random_fs -= fitness_score(sorted_population[i], adj_matrix)
        if random_fs < 0:
            return population[i]

In [44]:
# Define the order-based crossover function
def order_crossover(parent1, parent2):
    # Choose a random subset of genes from the first parent
    start = np.random.randint(len(parent1))
    end = np.random.randint(start, len(parent1))
    subset = parent1[start:end+1]
    # Initialize the child with the subset
    child = [-1] * len(parent1)
    child[start:end+1] = subset
    # Fill in the remaining genes from the second parent
    j = 0
    for i in range(len(parent1)):
        if not parent2[i] in subset:
            while child[j] != -1:
                j += 1
            child[j] = parent2[i]
    return child

In [84]:
def genetic_algorithm(sorted_population, adj_matrix):

    cross_over_perc = 0.8
    mutation_rate = 0.1
    child_population = []
    for _ in range(80):
        
        parent1 = random.choice(sorted_population)
        parent2 = random.choice(sorted_population)
        
        #parent1 = roulette(sorted_population, adj_matrix)
        #parent2 = roulette(sorted_population, adj_matrix)

        child = order_crossover(parent1, parent2)
        child_population.append(child)

    child_population = [mutate_path(child, mutation_rate) for child in child_population]
    new_population = child_population + sorted_population[:20]
    sorted_population = sorted(new_population, key = lambda x: fitness_score(x, adj_matrix), reverse = True)
    return sorted_population

In [13]:
def make_adj_matrix(city):
    adj_matrix = pd.DataFrame(index = [i for i in range(len(city))], columns = [i for i in range(len(city))])
    for i in range(len(city)):
        for j in range(len(city)): #set range(i+1) if you want to make a lower triangle
            lat1 = math.radians(city.iloc[i, 0])
            long1 = math.radians(city.iloc[i, 1])
            lat2 = math.radians(city.iloc[j, 0])
            long2 = math.radians(city.iloc[j,1])
            adj_matrix.iloc[i, j] = great_circle_dist(lat1, lat2, long1, long2)
    return np.array(adj_matrix)

In [10]:
def find_greedy_path(adj_matrix):
    visited_path = [0,]
    i = 0
    adj_matrix = pd.DataFrame(adj_matrix)
    for _ in range(len(adj_matrix)):
        adj_matrix = adj_matrix.sort_values(i)
        for idx in adj_matrix.index:
            if idx not in visited_path:
                visited_path.append(idx)
                i = idx
                break
                
    return visited_path

In [11]:
def plot_travel(optimal_path, city, plot_title):
    plt.plot(city.iloc[optimal_path,:].iloc[:,0], city.iloc[optimal_path,:].iloc[:,1])
    plt.title('{}'.format(plot_title))
    plt.show()

In [12]:
def generate_random_population(population_size, num_of_cities):
    population = []
    for sample in range(population_size):
        population.append(random.sample([i for i in range(num_of_cities)], num_of_cities))
    return population